In [ ]:
import pandas as pd
import numpy as np
from catnip.fla_redshift import FLA_Redshift
from sqlalchemy import null
from datetime import datetime

from prefect.blocks.system import Secret
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

In [1]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

NameError: name 'Dict' is not defined

In [ ]:
# LTP

In [ ]:
df = pd.read_csv("C:\\Users\\riffere\\OneDrive - Florida Panthers\\Documents\\LTP\\LTP_Combined.csv")
FLA_Redshift(**rs_creds).write_to_warehouse(df = df, table_name= "community_ltp_data")

In [ ]:
q = """
WITH initial AS (
    SELECT
        LOWER(parent_email) AS parent_email,
        MIN(season) AS season,
        MIN(date(transaction_date)) AS transaction_date
    FROM
        custom.community_ltp_data
    GROUP BY
        parent_email
),
acct_rev_info AS (
    SELECT
        contacts.email,
        createdon AS add_date,
        CASE
            WHEN ticket.season LIKE '2021-22' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2122",
        CASE
            WHEN ticket.season LIKE '2022-23' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2223",
        CASE
            WHEN ticket.season LIKE '2023-24' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2324",
        CASE
            WHEN ticket.season LIKE '2024-25' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2425",
        CASE
            WHEN ticket.season LIKE '2021-22' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2122",
        CASE
            WHEN ticket.season LIKE '2022-23' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2223",
        CASE
            WHEN ticket.season LIKE '2023-24' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2324",
        CASE
            WHEN ticket.season LIKE '2024-25' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2425"
    FROM
        custom.cth_v_historical_ticket ticket
    JOIN
        custom.korepss_externalsystemtocontact ext ON ticket.purchaser_ticketing_id = ext.externalcontactid
    JOIN
        custom.korepss_contacts contacts ON ext.crmcontactid = contacts.contactid
),
subs_2526 as (
    SELECT
        email,
        sum(gross_revenue) AS "gross_rev_2526",
        1 AS "is_stm_2526"
    FROM
        custom.cth_v_ticket_subscription_2526 subs
    INNER JOIN
        custom.korepss_externalsystemtocontact ext ON subs.purch_client_crm_id = ext.externalcontactid
    INNER JOIN
        custom.korepss_contacts contacts ON ext.crmcontactid = contacts.contactid
    GROUP BY
        email
),
acct_rev_agg AS (
    SELECT
        email,
        MIN(add_date) AS add_date,
        SUM(gross_rev_2122) AS gross_rev_2122,
        SUM(gross_rev_2223) AS gross_rev_2223,
        SUM(gross_rev_2324) AS gross_rev_2324,
        SUM(gross_rev_2425) AS gross_rev_2425,
        MAX(is_stm_2122) AS is_stm_2122,
        MAX(is_stm_2223) AS is_stm_2223,
        MAX(is_stm_2324) AS is_stm_2324,
        MAX(is_stm_2425) AS is_stm_2425
    FROM
        acct_rev_info
    GROUP BY
        email
)
SELECT
    parent_email,
    transaction_date,
    initial.season,
    add_date,
    COALESCE(gross_rev_2122, 0) AS gross_rev_2122,
    COALESCE(gross_rev_2223, 0) AS gross_rev_2223,
    COALESCE(gross_rev_2324, 0) AS gross_rev_2324,
    COALESCE(gross_rev_2425, 0) AS gross_rev_2425,
    COALESCE(gross_rev_2526, 0) AS gross_rev_2526,
    COALESCE(is_stm_2122, 0) AS is_stm_2122,
    COALESCE(is_stm_2223, 0) AS is_stm_2223,
    COALESCE(is_stm_2324, 0) AS is_stm_2324,
    COALESCE(is_stm_2425, 0) AS is_stm_2425,
    COALESCE(is_stm_2526, 0) AS is_stm_2526
FROM
    initial
LEFT JOIN
    acct_rev_agg ON initial.parent_email = acct_rev_agg.email
LEFT JOIN
    subs_2526 ON initial.parent_email = subs_2526.email
WHERE
    initial.parent_email IS NOT NULL
    AND initial.parent_email <> ''
    AND initial.season IN ('2021-22', '2022-23', '2023-24', '2024-25')
"""

In [ ]:
df = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [ ]:
df['add_date'].fillna(datetime.strptime('2030-01-01', "%Y-%m-%d"), inplace = True)
df['add_date'] = pd.to_datetime(df['add_date']).dt.date
df['transaction_date'] = pd.to_datetime(df['transaction_date']).dt.date

C:\Users\riffere\AppData\Local\Temp\ipykernel_30076\4015671550.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['add_date'].fillna(datetime.strptime('2030-01-01', "%Y-%m-%d"), inplace = True)


In [ ]:
df['pre_ltp_email'] = df.apply(lambda row: 0 if row['transaction_date'] == '' else (1 if row['add_date'] < row['transaction_date'] else 0), axis = 1)
df['post_ltp_email'] = df.apply(lambda row: 0 if row['transaction_date'] == '' else (1 if row['add_date'] >= row['transaction_date'] else 0), axis = 1)

In [ ]:
season_dict = {
    "2021-22" : 1,
    "2022-23" : 2,
    "2023-24" : 3,
    "2024-25" : 4
}

# sum season tickets before/after
for key, value in season_dict.items():
    col_list = list(df.columns[4:(4+value)])
    # print(col_list)
    df.loc[(df['season'] == key), 'pre_ltp_revenue'] =  df[col_list].sum(axis = 1)

for key, value in season_dict.items():
    col_list = list(df.columns[(4 + value):9])
    # print(col_list)
    df.loc[(df['season'] == key), 'post_ltp_revenue'] =  df[col_list].sum(axis = 1)

# sum revenue before/after
for key, value in season_dict.items():
    col_list = list(df.columns[9:(9+value)])
    # print(col_list)
    df.loc[(df['season'] == key), 'pre_ltp_tickets'] =  df[col_list].sum(axis = 1)


for key, value in season_dict.items():
    col_list = list(df.columns[(9 + value):14])
    # print(col_list)
    df.loc[(df['season'] == key), 'post_ltp_tickets'] =  df[col_list].sum(axis = 1)

In [ ]:
FLA_Redshift(**rs_creds).write_to_warehouse(df = df, table_name= "community_v_ltp")

In [ ]:
# PE FLOORBALL

In [ ]:
df = pd.read_csv("C:\\Users\\riffere\\OneDrive - Florida Panthers\\Documents\\LTP\\PE_Floorball_Combined.csv")
FLA_Redshift(**rs_creds).write_to_warehouse(df = df, table_name= "community_pe_floorball_data")

In [ ]:
q = """
WITH initial AS (
    SELECT
        LOWER(email) AS parent_email,
        MIN(season) AS season,
        MIN(date(transaction_date)) AS transaction_date
    FROM
        custom.community_pe_floorball_data
    GROUP BY
        parent_email
),
acct_rev_info AS (
    SELECT
        contacts.email,
        createdon AS add_date,
        CASE
            WHEN ticket.season LIKE '2021-22' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2122",
        CASE
            WHEN ticket.season LIKE '2022-23' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2223",
        CASE
            WHEN ticket.season LIKE '2023-24' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2324",
        CASE
            WHEN ticket.season LIKE '2024-25' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2425",
        CASE
            WHEN ticket.season LIKE '2021-22' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2122",
        CASE
            WHEN ticket.season LIKE '2022-23' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2223",
        CASE
            WHEN ticket.season LIKE '2023-24' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2324",
        CASE
            WHEN ticket.season LIKE '2024-25' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2425"
    FROM
        custom.cth_v_historical_ticket ticket
    JOIN
        custom.korepss_externalsystemtocontact ext ON ticket.purchaser_ticketing_id = ext.externalcontactid
    JOIN
        custom.korepss_contacts contacts ON ext.crmcontactid = contacts.contactid
),
subs_2526 as (
    SELECT
        email,
        sum(gross_revenue) AS "gross_rev_2526",
        1 AS "is_stm_2526"
    FROM
        custom.cth_v_ticket_subscription_2526 subs
    INNER JOIN
        custom.korepss_externalsystemtocontact ext ON subs.purch_client_crm_id = ext.externalcontactid
    INNER JOIN
        custom.korepss_contacts contacts ON ext.crmcontactid = contacts.contactid
    GROUP BY
        email
),
acct_rev_agg AS (
    SELECT
        email,
        MIN(add_date) AS add_date,
        SUM(gross_rev_2122) AS gross_rev_2122,
        SUM(gross_rev_2223) AS gross_rev_2223,
        SUM(gross_rev_2324) AS gross_rev_2324,
        SUM(gross_rev_2425) AS gross_rev_2425,
        MAX(is_stm_2122) AS is_stm_2122,
        MAX(is_stm_2223) AS is_stm_2223,
        MAX(is_stm_2324) AS is_stm_2324,
        MAX(is_stm_2425) AS is_stm_2425
    FROM
        acct_rev_info
    GROUP BY
        email
)
SELECT
    parent_email,
    transaction_date,
    initial.season,
    add_date,
    COALESCE(gross_rev_2122, 0) AS gross_rev_2122,
    COALESCE(gross_rev_2223, 0) AS gross_rev_2223,
    COALESCE(gross_rev_2324, 0) AS gross_rev_2324,
    COALESCE(gross_rev_2425, 0) AS gross_rev_2425,
    COALESCE(gross_rev_2526, 0) AS gross_rev_2526,
    COALESCE(is_stm_2122, 0) AS is_stm_2122,
    COALESCE(is_stm_2223, 0) AS is_stm_2223,
    COALESCE(is_stm_2324, 0) AS is_stm_2324,
    COALESCE(is_stm_2425, 0) AS is_stm_2425,
    COALESCE(is_stm_2526, 0) AS is_stm_2526
FROM
    initial
LEFT JOIN
    acct_rev_agg ON initial.parent_email = acct_rev_agg.email
LEFT JOIN
    subs_2526 ON initial.parent_email = subs_2526.email
WHERE
    initial.parent_email IS NOT NULL 
    AND initial.parent_email <> ''
    AND initial.season IN ('2023-24', '2024-25')
"""

In [ ]:
df = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [ ]:
df['add_date'].fillna(datetime.strptime('2030-01-01', "%Y-%m-%d"), inplace = True)
df['add_date'] = pd.to_datetime(df['add_date']).dt.date
df['transaction_date'] = pd.to_datetime(df['transaction_date']).dt.date

C:\Users\riffere\AppData\Local\Temp\ipykernel_30076\4015671550.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['add_date'].fillna(datetime.strptime('2030-01-01', "%Y-%m-%d"), inplace = True)


In [ ]:
df['pre_pe_floorball_email'] = df.apply(lambda row: 0 if row['transaction_date'] == '' else (1 if row['add_date'] < row['transaction_date'] else 0), axis = 1)
df['post_pe_floorball_email'] = df.apply(lambda row: 0 if row['transaction_date'] == '' else (1 if row['add_date'] >= row['transaction_date'] else 0), axis = 1)

In [ ]:
season_dict = {
    "2021-22" : 1,
    "2022-23" : 2,
    "2023-24" : 3,
    "2024-25" : 4
}

# sum season tickets before/after
for key, value in season_dict.items():
    col_list = list(df.columns[4:(4+value)])
    # print(col_list)
    df.loc[(df['season'] == key), 'pre_pe_floorball_revenue'] =  df[col_list].sum(axis = 1)

for key, value in season_dict.items():
    col_list = list(df.columns[(4 + value):9])
    # print(col_list)
    df.loc[(df['season'] == key), 'post_pe_floorball_revenue'] =  df[col_list].sum(axis = 1)

# sum revenue before/after
for key, value in season_dict.items():
    col_list = list(df.columns[9:(9+value)])
    # print(col_list)
    df.loc[(df['season'] == key), 'pre_pe_floorball_tickets'] =  df[col_list].sum(axis = 1)


for key, value in season_dict.items():
    col_list = list(df.columns[(9 + value):14])
    # print(col_list)
    df.loc[(df['season'] == key), 'post_pe_floorball_tickets'] =  df[col_list].sum(axis = 1)

In [ ]:
FLA_Redshift(**rs_creds).write_to_warehouse(df = df, table_name= "community_v_pe_floorball")

In [ ]:
# LTP Ball Hockey

In [ ]:
df = pd.read_csv("C:\\Users\\riffere\\OneDrive - Florida Panthers\\Documents\\LTP\\LTP_Ball_Hockey_Combined.csv")
FLA_Redshift(**rs_creds).write_to_warehouse(df = df, table_name= "community_ltp_ball_hockey_data")

In [ ]:
q = """
WITH initial AS (
    SELECT
        LOWER(email) AS parent_email,
        MIN(season) AS season,
        MIN(date(transaction_date)) AS transaction_date
    FROM
        custom.community_ltp_ball_hockey_data
    GROUP BY
        parent_email
),
acct_rev_info AS (
    SELECT
        contacts.email,
        createdon AS add_date,
        CASE
            WHEN ticket.season LIKE '2021-22' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2122",
        CASE
            WHEN ticket.season LIKE '2022-23' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2223",
        CASE
            WHEN ticket.season LIKE '2023-24' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2324",
        CASE
            WHEN ticket.season LIKE '2024-25' THEN gross_revenue
            ELSE 0
        END AS "gross_rev_2425",
        CASE
            WHEN ticket.season LIKE '2021-22' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2122",
        CASE
            WHEN ticket.season LIKE '2022-23' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2223",
        CASE
            WHEN ticket.season LIKE '2023-24' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2324",
        CASE
            WHEN ticket.season LIKE '2024-25' AND "ticket_type" IN ('Annual Suites', 'Full', 'Half', 'Premier') THEN 1
            ELSE 0
        END AS "is_stm_2425"
    FROM
        custom.cth_v_historical_ticket ticket
    JOIN
        custom.korepss_externalsystemtocontact ext ON ticket.purchaser_ticketing_id = ext.externalcontactid
    JOIN
        custom.korepss_contacts contacts ON ext.crmcontactid = contacts.contactid
),
subs_2526 as (
    SELECT
        email,
        sum(gross_revenue) AS "gross_rev_2526",
        1 AS "is_stm_2526"
    FROM
        custom.cth_v_ticket_subscription_2526 subs
    INNER JOIN
        custom.korepss_externalsystemtocontact ext ON subs.purch_client_crm_id = ext.externalcontactid
    INNER JOIN
        custom.korepss_contacts contacts ON ext.crmcontactid = contacts.contactid
    GROUP BY
        email
),
acct_rev_agg AS (
    SELECT
        email,
        MIN(add_date) AS add_date,
        SUM(gross_rev_2122) AS gross_rev_2122,
        SUM(gross_rev_2223) AS gross_rev_2223,
        SUM(gross_rev_2324) AS gross_rev_2324,
        SUM(gross_rev_2425) AS gross_rev_2425,
        MAX(is_stm_2122) AS is_stm_2122,
        MAX(is_stm_2223) AS is_stm_2223,
        MAX(is_stm_2324) AS is_stm_2324,
        MAX(is_stm_2425) AS is_stm_2425
    FROM
        acct_rev_info
    GROUP BY
        email
)
SELECT
    parent_email,
    transaction_date,
    initial.season,
    add_date,
    COALESCE(gross_rev_2122, 0) AS gross_rev_2122,
    COALESCE(gross_rev_2223, 0) AS gross_rev_2223,
    COALESCE(gross_rev_2324, 0) AS gross_rev_2324,
    COALESCE(gross_rev_2425, 0) AS gross_rev_2425,
    COALESCE(gross_rev_2526, 0) AS gross_rev_2526,
    COALESCE(is_stm_2122, 0) AS is_stm_2122,
    COALESCE(is_stm_2223, 0) AS is_stm_2223,
    COALESCE(is_stm_2324, 0) AS is_stm_2324,
    COALESCE(is_stm_2425, 0) AS is_stm_2425,
    COALESCE(is_stm_2526, 0) AS is_stm_2526
FROM
    initial
LEFT JOIN
    acct_rev_agg ON initial.parent_email = acct_rev_agg.email
LEFT JOIN
    subs_2526 ON initial.parent_email = subs_2526.email
WHERE
    initial.parent_email IS NOT NULL 
    AND initial.parent_email <> ''
    AND initial.season IN ('2023-24', '2024-25')
"""

In [ ]:
df = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [ ]:
df['add_date'].fillna(datetime.strptime('2030-01-01', "%Y-%m-%d"), inplace = True)
df['add_date'] = pd.to_datetime(df['add_date']).dt.date
df['transaction_date'] = pd.to_datetime(df['transaction_date']).dt.date

C:\Users\riffere\AppData\Local\Temp\ipykernel_30076\4015671550.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['add_date'].fillna(datetime.strptime('2030-01-01', "%Y-%m-%d"), inplace = True)


In [ ]:
df['pre_ltp_ball_hockey_email'] = df.apply(lambda row: 0 if row['transaction_date'] == '' else (1 if row['add_date'] < row['transaction_date'] else 0), axis = 1)
df['post_ltp_ball_hockey_email'] = df.apply(lambda row: 0 if row['transaction_date'] == '' else (1 if row['add_date'] >= row['transaction_date'] else 0), axis = 1)

In [ ]:
season_dict = {
    "2021-22" : 1,
    "2022-23" : 2,
    "2023-24" : 3,
    "2024-25" : 4
}

# sum season tickets before/after
for key, value in season_dict.items():
    col_list = list(df
                    
                    .columns[4:(4+value)])
    # print(col_list)
    df.loc[(df['season'] == key), 'pre_ltp_ball_hockey_revenue'] =  df[col_list].sum(axis = 1)

for key, value in season_dict.items():
    col_list = list(df.columns[(4 + value):9])
    # print(col_list)
    df.loc[(df['season'] == key), 'post_ltp_ball_hockey_revenue'] =  df[col_list].sum(axis = 1)

# sum revenue before/after
for key, value in season_dict.items():
    col_list = list(df.columns[9:(9+value)])
    # print(col_list)
    df.loc[(df['season'] == key), 'pre_ltp_ball_hockey_tickets'] =  df[col_list].sum(axis = 1)


for key, value in season_dict.items():
    col_list = list(df.columns[(9 + value):14])
    # print(col_list)
    df.loc[(df['season'] == key), 'post_ltp_ball_hockey_tickets'] =  df[col_list].sum(axis = 1)

In [ ]:
df

,parent_email,transaction_date,season,add_date,gross_rev_2122,gross_rev_2223,gross_rev_2324,gross_rev_2425,gross_rev_2526,is_stm_2122,...,is_stm_2324,is_stm_2425,is_stm_2526,pre_ltp_ball_hockey_email,post_ltp_ball_hockey_email,pre_ltp_ball_hockey_revenue,post_ltp_ball_hockey_revenue,pre_ltp_ball_hockey_tickets,post_ltp_ball_hockey_tickets,processed_date
0,kmatta90@gmail.com,2024-08-12,2024-25,2021-10-26,432.0,0.0,0.00,0.00,0.0,0,...,0,0,0,1,0,432.00,0.0,0.0,0.0,2025-01-29 17:14:47.659249+00:00
1,jessicavones@gmail.com,2025-01-25,2024-25,2018-01-22,0.0,0.0,0.00,0.00,0.0,0,...,0,0,0,1,0,0.00,0.0,0.0,0.0,2025-01-29 17:14:47.659249+00:00
2,mawright3002@gmail.com,2024-08-08,2024-25,2023-03-03,0.0,145.0,187.25,0.00,0.0,0,...,0,0,0,1,0,332.25,0.0,0.0,0.0,2025-01-29 17:14:47.659249+00:00
3,dianafuentesmom2@gmail.com,2024-08-06,2024-25,2030-01-01,0.0,0.0,0.00,0.00,0.0,0,...,0,0,0,0,1,0.00,0.0,0.0,0.0,2025-01-29 17:14:47.659249+00:00
4,ervf321@yahoo.com,2024-08-07,2024-25,2030-01-01,0.0,0.0,0.00,0.00,0.0,0,...,0,0,0,0,1,0.00,0.0,0.0,0.0,2025-01-29 17:14:47.659249+00:00
5,amandac617@hotmail.com,2025-01-24,2024-25,2030-01-01,0.0,0.0,0.00,0.00,0.0,0,...,0,0,0,0,1,0.00,0.0,0.0,0.0,2025-01-29 17:14:47.659249+00:00
6,tg7tinkerbell@gmail.com,2024-08-08,2024-25,2030-01-01,0.0,0.0,0.00,0.00,0.0,0,...,0,0,0,0,1,0.00,0.0,0.0,0.0,2025-01-29 17:14:47.659249+00:00
7,oslynrodriguez@gmail.com,2024-08-05,2024-25,2030-01-01,0.0,0.0,0.00,0.00,0.0,0,...,0,0,0,0,1,0.00,0.0,0.0,0.0,2025-01-29 17:14:47.659249+00:00
8,02lisamm@gmail.com,2024-08-12,2024-25,2030-01-01,0.0,0.0,0.00,0.00,0.0,0,...,0,0,0,0,1,0.00,0.0,0.0,0.0,2025-01-29 17:14:47.659249+00:00
9,shelby@sunshinehoist.com,2025-01-24,2024-25,2030-01-01,0.0,0.0,0.00,0.00,0.0,0,...,0,0,0,0,1,0.00,0.0,0.0,0.0,2025-01-29 17:14:47.659249+00:00


In [26]:
FLA_Redshift(**rs_creds).write_to_warehouse(df = df, table_name= "community_v_ltp_ball_hockey")